# What is about ? PRELIMINARY DRAFT TO BE MODIFIED 



Notebook for https://www.kaggle.com/alsgroup/end-als

We consider transcriptomics data and make some EDA (Explanatory Data Analysis).
Transcriptomics - measures genes expressions for cells. There are 53859 genes.


Some findings: 

1) We see data are non-negative - that means probably they obtained - log(1+counts)+normalization(?) - typical processing.

2) 173 genes have zero expression - we can throw them out

3) Top expressed genes contain MALAT1 , MT.*** (Mitochondrial genes ( https://en.wikipedia.org/wiki/Mitochondrial_DNA )) 
that seems to happen for many transcriptomics datasets, so it  seems  not worth for analysis.

(See for example https://www.kaggle.com/alexandervc/sciplex2-part-1-eda?scriptVersionId=57667001 cell 22 - MALAT1 abd MT.RNR2 - on top , or https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html cell 8 - MALAT1 - on top )

4) Gene NEFL - "is a useful marker for disease monitoring in amyotrophic lateral sclerosis,[9]" (among top expressed - does it mean something ?)
https://en.wikipedia.org/wiki/Neurofilament_light_polypeptide

It is one the top expressed genes - and related to neuro diseases.

 Roc auc on three datasets: 
 bulbar_vs_limb 	median_low_vs_high  ctrl_vs_case 
NEFL	0.392	0.397	0.489


5) List of selected genes and their roc auc

These genes taken from wikipedia and etc.:

list_genes = ['NEFL', 'C9orf72', 'SOD1',  'TARDBP','FUS', 'NEFH',
              'MATR3', 'TUBA4A' ,
'ANXA11', 'NEK1',  'TBK1', 'CCNF', 'TIA1',
             'ALS2']
             
We check their roc_auc. Beware that std for completely random predictor is about 0.05, so no gene goes beyond 3 std, and thus by its own can be considered reliably good predictor, however - may be good prediction on all three targets simultaneously would mean something - I will check later.


6) Correlation analysis. We see that FUS is strongly anticorrelated with many genes.
Genes: ['C9orf72','MATR3','TUBA4A','NEFL'] correlated with each other about 0.3-0.5 and anticorrelated with FUS about 0.5-0.6.
Such correletions seems to cannot be random. 


7) Added info on genes from mygene package. Surprised that cell cycle genes CCNF, NEK1 are also related to ALS.
NEK1 - Associated with 3-5% of ALS cases; **considered an ALS risk gene rather than a causative gene** as of 2018.[1]
( https://en.wikipedia.org/wiki/Genetics_of_amyotrophic_lateral_sclerosis  )

ANXA11 is quite correlated with CCNF. While ANXA11 - 'Definite ALS' (and also schizophrenia ) 


8) Dimensional reduction by PCA, UMAP does not reveal clear clusters in data, neither separation by targers is not seen. 

9) Added: analysis of strictly bigger genes list ( 154  ALS related genes) from https://alsod.ac.uk/ 

9.1) We see correlation are much higher than for random data or even for random subset of genes from our dataset.
That suggests that this subset is not kind of random.

9.2) Top genes by predictive power - OPTN HNRNPA2B1 have RocAuc on second task about 65 which is of order 3* std of random,
which is not bad evidence that it makes sense. (OPTN is marked as "Definitive ALS gene" on site above, while HNRNPA2B1 as "Tenuous".

9.3) Moreover these genes are highly (anti)-correlated with other known ALS genes: FUS, NEFL and with each other - yet another evidence.

9.4) Top 3 - NIPA1  - marked "Strong evidence" - strongly (anti)-correlated with NEFL,OPTN - which are known to be related to ALS, also 0.37 correlated with SOD1 - definitive ALS, and 0.68 with SOD2 - tenous , while SOD1,SOD2 are correlated and by definition related.
NIPA1 - is the best predict for bulbar_vs_limb.

9.5) ANXA11 - 'Definite ALS' (and also schizophrenia ) - is highly correlated with many other relevant genes  like NEFL,KIF5A.
It seems kind of heart of the high correlation cluster. 



PSPS


https://en.wikipedia.org/wiki/Amyotrophic_lateral_sclerosis

https://en.wikipedia.org/wiki/Amyotrophic_lateral_sclerosis_research

https://en.wikipedia.org/wiki/Genetics_of_amyotrophic_lateral_sclerosis



-------------------

Version 10: mild updates - from 154 genes some are not or very poorly expressed in current data - probably should be excluded.

Version 9: Added: analysis of strictly bigger genes list ( 154  ALS related genes) from https://alsod.ac.uk/ 

Version 8: Added: Dimensional reductions PCA, UMAP and visualization

Version 7: Added: information on genes with mygene package 

Version 6:  Added: Selected genes correlation analysis

Version 5: Added: selected genes list and checked their roc_auc


Not mentioned version - only cosmetic changes. 


Should be more analyzed later: 



# Gene NEFL -  "is a useful marker for disease monitoring in amyotrophic lateral sclerosis,[9]" (among top expressed - does it mean something ?)

https://en.wikipedia.org/wiki/Neurofilament_light_polypeptide

It is one the top expressed genes - and related to neuro diseases.


What is surprising - its relation to ALS mentioned here: https://en.wikipedia.org/wiki/Neurofilament_light_polypeptide
but not mentioned here - https://en.wikipedia.org/wiki/Genetics_of_amyotrophic_lateral_sclerosis
where it must be mentioned - special page containing genes associated to ALS




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        i += 1
        if i < 5:
            print(os.path.join(dirname, filename))
print('Printed 5 filenames out of ', i)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load largest transcriptomics data

In [ ]:
# fn = '/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv'
# fn = '/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv'
fn = '/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv'
df = pd.read_csv(fn)
df

# EDA

In [ ]:
y = df.iloc[:,1]
df_save = df.copy()
df = df.iloc[:,2:] # Keep only genes, no target and ids
print('Keep only genes, no target and ids')


In [ ]:
print('Check no duplicated gene names:')
len(set(df.columns) ) , len(df.columns)

In [ ]:
print('Check no negative values:')
(df < 0 ).sum().sum()

In [ ]:
print('Look on number of zero expressed genes:')
v1 = df.sum(axis = 0)
print((v1==0).sum())
v2 = df.std(axis = 0)
print((v2==0).sum())


In [ ]:
print('Look on top expressed and lowest expressed genes')
genes = pd.DataFrame()
genes['Expression Sum'] = v1
genes['Expression Mean'] = v1/len(v1)
genes['Expression Std'] = v2
genes['Index'] = range(len(genes)) # 
genes = genes.sort_values('Expression Sum',ascending = False)
display(genes.head(20))
print()
display(genes.tail(20))



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize = (20,6))
fig.add_subplot(1,2,1)
c = 'Expression Mean'
plt.plot(np.log10(1+ genes[c].values) , '*-')
plt.title(c + ' Log10 scale')
plt.xlabel('genes ordered')
fig.add_subplot(1,2,2)
c = 'Expression Std'
plt.plot(np.log10(1+ genes['Expression Std'].values) , '*-')
plt.title(c + ' Log10 scale')
plt.xlabel('genes ordered')
plt.show()
print('More or less ordering by expression mean is the same as by std - that is natural')
print('Typically one may keep top variable genes to analyse')

# NEFL gene and other genes known from literature  (see bigger list below)

Remark. Here we look on list of genes from wikipedia+- , in the subsequent section on greate list from https://alsod.ac.uk/


From challenge introduction: 
"The C9orf72, SOD1, TARDBP, and FUS genes are key to the normal functioning of motor neurons and other cells."

See table https://en.wikipedia.org/wiki/Genetics_of_amyotrophic_lateral_sclerosis
but it seems to be not quite complete, even information on  the same page in history section about NEFH is not table (!).


https://en.wikipedia.org/wiki/Genetics_of_amyotrophic_lateral_sclerosis#History

The first gene to be associated with ALS was SOD1, which was identified in 1993. It was the first time that linkage analysis was successful in identifying the genetic cause of a rare neurodegenerative disorder.[6] SOD1 is one of the most common genes associated with ALS, accounting for about 12% of fALS and 1-2% of sALS. The second gene, NEFH, was identified in 1994, followed by SETX in 1998, ALS2 in 2001, DCTN1 in 2003, and CHMP2B in 2006. All of these genes are fairly rare; the next major ALS gene, TARDBP, was identified in 2008 and accounts for 4% of fALS and 1% of sALS. FUS was identified in 2009 and is seen in 4% of fALS and 1% of sALS. VCP was identified in 2010 and accounts for 1% of fALS and 1% of sALS; ATXN2, OPTN, and UBQLN2 were associated with ALS that same year.[2]

Another major milestone was the discovery of C9orf72 in 2011, which is the most common gene associated with ALS, accounting for about 40% of fALS cases and 7% of sALS cases. C9orf72 was also found to contribute significantly to frontotemporal dementia (FTD). SQSTM1 was also identified in 2011, but accounts for 1% of fALS and less than 1% of sALS. PFN1 was identified in 2012, HNRNPA1 and HNRNPA2B1 in 2013, CHCHD10, MATR3, and TUBA4A in 2014, and TBK1 in 2015. C21orf2, CCNF, and NEK1 were associated with ALS in 2016.[2]

The first genome-wide association study (GWAS) of ALS was published in 2007, and 14 GWASs total had been published through 2013. They have contributed significantly to our understanding of ALS genetics; for example, a 2010 GWAS studying ALS in Finland led to discovery of the role of mutations at the C9orf72 locus in ALS. However, a gene identified by a single GWAS may not actually be associated with ALS, especially if the cohort size is small. In outbred populations, thousands of cases (people with ALS) and controls (people without ALS) are required for a GWAS to have sufficient statistical power to confidently identify a gene's association with ALS.[6]

In [ ]:
list_genes = ['NEFL', 'C9orf72', 'SOD1',  'TARDBP','FUS', 'NEFH',
              'MATR3', 'TUBA4A' ,
'ANXA11', 'NEK1',  'TBK1', 'CCNF', 'TIA1',
             'ALS2']

# 'C21orf2', - not found 

for g in list_genes:
    if g not in df.columns:
        print(g , 'not found in given genes')
    else:
        print(g, 'found')

In [ ]:
from sklearn.metrics import roc_auc_score

u = df['NEFL'].values
r = roc_auc_score(y.values,u)
print(r)

In [ ]:
list_fn = ['/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv', 
'/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv',
'/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv']
import time
t0 = time.time()
list_df = []
for fn in list_fn:
    d = pd.read_csv(fn)
    list_df.append(d)
list_names = [ 'bulbar_vs_limb', 'median_low_vs_high',  'ctrl_vs_case' ]

print(time.time()-t0, 'seconds passed')

In [ ]:
st = pd.DataFrame()

for i,d in enumerate(list_df):
    for c in list_genes: 
        IX = list_names[i]
        u = d[c].values
        y = d.iloc[:,1].values
        r = roc_auc_score(y,u)
        
        st.loc[ c, IX +' RocAuc' ] = np.round(r,3) 
st.index.name = 'gene'



print('RocAuc for selected genes (known from previous literature)')
st    

# Selected genes correlation analysis

In [ ]:
import seaborn as sns
cm = df[list_genes].corr()
#correlation heatmap
plt.figure(figsize=(10,6));
sns.heatmap(cm);

In [ ]:
#clustermap
sns.clustermap(np.abs(cm),cmap='vlag');

In [ ]:
df[['FUS','C9orf72','MATR3','TUBA4A','NEFL']].corr()

In [ ]:
a,b = np.where( np.triu(np.abs(cm),1) > 0.3 )

d = pd.DataFrame()
for i in range(len(a)):
    d.loc[i,'Gene1'] = cm.index[a[i]] 
    d.loc[i,'Gene2'] = cm.index[b[i]] 
    d.loc[i,'Correlation'] = cm.iloc[a[i],b[i]]
    d.loc[i,'Correlation Abs'] = np.abs( cm.iloc[a[i],b[i]] )
    
d.sort_values('Correlation Abs',ascending = False).head(30)

In [ ]:
cm['FUS']

In [ ]:
v = np.triu((cm),1).ravel()
v = v[v != 0]
plt.hist(v,bins = 5)
plt.show()

# Genes info with mygene package

mygeny - is a nice package to get info on genes
See example notebook: 
https://www.kaggle.com/alexandervc/genes-info-with-mygene-python-package


In [ ]:
!pip install mygene
import mygene
mg = mygene.MyGeneInfo()

In [ ]:
mg = mygene.MyGeneInfo()
import time
#genes = df.copy()
genes = pd.DataFrame()
t0 = time.time()

for g in list_genes:
    g_inf = mg.query('symbol:'+g, species='human')
    #print(g, g_inf)
    if 'hits' in g_inf.keys():
        if 'entrezgene' in g_inf['hits'][0].keys(): 
            IX = g
            entrez_gene_ID = g_inf['hits'][0]['entrezgene' ]

            g_inf = mg.getgene(entrez_gene_ID)
            list1 = ['symbol',   'type_of_gene', 'map_location' ,   'name', 'entrezgene', 'alias', 'other_names' , 'summary' ] 
            for k in list1:
                if k in g_inf.keys():
                    genes.loc[IX,k] = str(g_inf[k]) 

            if 'generif' in g_inf.keys():
                genes.loc[IX,'count pubmed'] = len(g_inf['generif'])            
            
genes 

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [ ]:
genes[['summary', 'count pubmed']].sort_values('count pubmed',ascending = False)

# Dimensional reduction and visualization

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA

t0 = time.time()

fig = plt.figure(figsize = (20,8) ); c = 0
for i,d in enumerate(list_df):
    data_name = list_names[i]
    X = d.iloc[:,2:].values
    y = d.iloc[:,1].values

    c += 1; fig.add_subplot(1,3,c)
    pca = PCA()# n_components=2)
    r = pca.fit_transform(X)
    
    sns.scatterplot( r[:,0],r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    print(time.time() - t0,'seconds passed')
    print(pca.explained_variance_ratio_[:10])
    print(pca.singular_values_[:10])

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


# UMAP visualization

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA

t0 = time.time()

fig = plt.figure(figsize = (20,8) ); c = 0
for i,d in enumerate(list_df):
    data_name = list_names[i]
    X = d.iloc[:,2:].values
    y = d.iloc[:,1].values

    c += 1; fig.add_subplot(1,3,c)
    pca = PCA()# n_components=2)
    r = pca.fit_transform(X)
    r = umap.UMAP().fit_transform( r[:,:100] )

    sns.scatterplot( r[:,0],r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.legend()
    
    print(time.time() - t0,'seconds passed')
    print(pca.explained_variance_ratio_[:10])
    print(pca.singular_values_[:10])

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


# Extended  list  of known genes related to ALS from https://alsod.ac.uk/ 

154 genes, with some comment on strongness of evidence. (Strictly contains previous one). 

149 of them are in our genes names for our data.

In [ ]:
url = 'https://raw.githubusercontent.com/chervov/genes/main/genes_ALS_from_alsod_ac_uk.csv'
df_genes_alsod = pd.read_csv(url)

if 0: # If inet is not available: 
    list_definitive_ALS_gene = ['ANXA11' ,'C9orf72' ,'CHCHD10' ,'EPHA4' ,'FUS' ,'HNRNPA1' ,'KIF5A' ,'NEK1' ,'OPTN' ,'PFN1' ,'SOD1' ,'TARDBP' ,'TBK1' ,'UBQLN2' ,'UNC13A' ,'VAPB' ,'VCP' ] 
    print(len( list_definitive_ALS_gene  ))
    list_strong_evidence_ALS_gene = ['ATXN1','CCNF','CFAP410','HFE','NIPA1','SCFD1','TUBA4A']
    print(len( list_strong_evidence_ALS_gene  ))
    list_clinical_modifier_ALS_gene =  ['ATXN2','CAMTA1','ENAH']
    print(len( list_clinical_modifier_ALS_gene  ))
    list_moderate_evidence_ALS_gene =  ['ANG','ARHGEF28','CDH22','CHMP2B','CNTN6','CRYM','CSNK1G3','CX3CR1','DAO','DNAJC7','DNMT3A','ERBB4','FIG4','GLE1','GPX3','LMNB1','SARM1','SMN1','SQSTM1','SS18L1','TNIP1']
    print(len( list_moderate_evidence_ALS_gene  ))
    list_tenuous_ALS_gene = ['ALS2', 'ALS3', 'ALS7', 'APEX1', 'APOE', 'AR', 'ARPP21', 'B4GALT6', 'BCL11B', 'BCL6', 'CCS', 'CDH13', 'CHGB', 'CNTF', 'CNTN4', 'CRIM1', 'CST3', 'CYP2D6', 'DCTN1', 'DIAPH3', 'DISC1', 'DNMT3B', 'DOC2B', 'DPP6', 'DYNC1H1', 'EFEMP1', 'ELP3', 'EphA3', 'ERLIN1', 'EWSR1', 'FEZF2', 'FGGY', 'GARS', 'GLT8D1', 'GRB14', 'GRN', 'HEXA', 'HNRNPA2B1', 'ITPR2', 'KDR', 'KIFAP3', 'LIF', 'LIPC', 'LOX', 'LUM', 'MAOB', 'MAPT', 'MATR3', 'MOBP', 'MTND2P1', 'NAIP', 'NEFH', 'NEFL', 'NETO1', 'NT5C1A', 'ODR4', 'OGG1', 'OMA1', 'PARK7', 'PCP4', 'PLEKHG5', 'PNPLA6', 'PON1', 'PON2', 'PON3', 'PRPH', 'PSEN1', 'PVR', 'RAMP3', 'RBMS1', 'RFTN1', 'RNASE2', 'RNF19A', 'SCN7A', 'SELL', 'SEMA6A', 'SETX', 'SIGMAR1', 'SLC1A2', 'SLC39A11', 'SLC52A3', 'SMN2', 'SNCG', 'SOD2', 'SOX5', 'SPAST', 'SPG11', 'SPG7', 'SUSD1', 'SYNE1', 'SYT9', 'TAF15', 'TIA1', 'TRPM7', 'UBQLN1', 'VDR', 'VEGFA', 'VPS54', 'VRK1', 'ZFP64', 'ZNF512B']
    print(len(list_tenuous_ALS_gene))
    # len(set(list_definitive_ALS_gene+list_strong_evidence_ALS_gene+list_clinical_modifier_gene+list_moderate_evidence_gene) & set(list_genes)), len(list_genes)

print(df_genes_alsod['Category'] .value_counts()    )
df_genes_alsod    

In [ ]:
print( set(list_genes) - set(df_genes_alsod['Gene symbol']), len( set(list_genes) ), len( df_genes_alsod ) )
print('Check - new genes list contains previus wikipedia list')

In [ ]:
for c in range(len(df_genes_alsod)): 
    g = df_genes_alsod['Gene symbol'].iat[c]
    IX_genes =  df_genes_alsod.index[c]
    list_roc_aucs = []
    for i,d in enumerate(list_df):
        data_name = list_names[i] # dataset name 
        if g in d.columns:
            df_genes_alsod.loc[ IX_genes, 'In Data' ] = True 
            u = d[g].values
            y = d.iloc[:,1].values
            r = roc_auc_score(y,u)
            df_genes_alsod.loc[ IX_genes, data_name +' RocAuc' ] = np.round(r,3) 
            
            list_roc_aucs.append(r)
        else:
            df_genes_alsod.loc[ IX_genes, 'In Data' ] = False 

    if len( list_roc_aucs ) > 0:
        best_abs_roc_auc = np.max( np.abs( np.array(list_roc_aucs) - 0.5) + 0.5 )
        df_genes_alsod.loc[ IX_genes, 'Best Abs RocAuc' ] =  np.round(best_abs_roc_auc,3)

df_genes_alsod['On Wiki'] = df_genes_alsod['Gene symbol'].isin(list_genes)        
df_genes_alsod.sort_values('Best Abs RocAuc', ascending = False ).head(50)

In [ ]:
mask = df_genes_alsod['Category'] == 'Definitive ALS gene'
df_genes_alsod[mask].sort_values('Best Abs RocAuc', ascending = False ).head(50)

In [ ]:
print('Show top predictors for each dataset')
for f  in [ 'bulbar_vs_limb', 'median_low_vs_high', 'ctrl_vs_case' ] :
    f2 = f + ' RocAuc'
    df_genes_alsod2 = df_genes_alsod.copy() 
    df_genes_alsod2[f2 + ' Abs'] =  ((df_genes_alsod[f2] - 0.5).abs() + 0.5)
    print('Top predictors for ',f)
    display( df_genes_alsod2.sort_values( f2 + ' Abs', ascending = False ).head(10) )


In [ ]:
l = list(df_genes_alsod.sort_values('Best Abs RocAuc', ascending = False ).head(5)['Gene symbol'] )
print('Correlations for top 5 predictors. We see that top2 - OPTN and HNRNPA2B1 are quite correlated')
df[l].corr()

In [ ]:
mask = df_genes_alsod['In Data'] == True
print(mask.sum())
l = list( df_genes_alsod['Gene symbol'][mask])
print( df[l].corr()['OPTN'].sort_values() )
print()
print('Conclusion:')
print('Top 1 - OPTN - strongly (anti)-correlated with FUS NEFL - which are known to be related to ALS ')
print('It support that its predictions are reasonable')

In [ ]:
mask = df_genes_alsod['In Data'] == True
l = list( df_genes_alsod['Gene symbol'][mask])
print( df[l].corr()['HNRNPA2B1'].sort_values() )
print()
print('Conclusion:')
print('Top 2 - HNRNPA2B1 - strongly (anti)-correlated with OPTN NEFL - which are known to be related to ALS ')
print('It support that its predictions are reasonable')

In [ ]:
mask = df_genes_alsod['In Data'] == True
l = list( df_genes_alsod['Gene symbol'][mask])
print( df[l].corr()['NIPA1'].sort_values().head(10) )
print('TAIL -------------------------------------------------')
print( df[l].corr()['NIPA1'].sort_values().tail(15) )
print()
print('Conclusion:')
print('Top 3 - NIPA1 - strongly (anti)-correlated with NEFL - which are known to be related to ALS, also 0.37 correlated with SOD1 - definitive ALS, and 0.68 with SOD2 - tenous ')
print('It support that its predictions are reasonable')
df[['NIPA1','SOD1','SOD2']].corr()

In [ ]:

mask = df_genes_alsod['In Data'] == True
l = list( df_genes_alsod['Gene symbol'][mask])
print( df[l].corr()['ANXA11'].sort_values().head(10) )
print('TAIL -------------------------------------------------')
print( df[l].corr()['ANXA11'].sort_values().tail(15) )
print()
print('Conclusion:')
print(' ANXA11 - strongly (anti)-correlated with NEFL - which are known to be related to ALS, also 0.37 correlated with SOD1 - definitive ALS, and 0.68 with SOD2 - tenous ')
print('It support that its predictions are reasonable')
df[['ANXA11','SOD1','SOD2']].corr()

In [ ]:
mask = (df_genes_alsod['Best Abs RocAuc'] > 0.59 )
mask.sum()
l = list( df_genes_alsod[mask]['Gene symbol'])
cm = df[l].corr()
import seaborn as sns
sns.heatmap(cm)
plt.show()

In [ ]:
mask = (df_genes_alsod['Best Abs RocAuc'] > 0.59 )
mask.sum()
l = list( df_genes_alsod[mask]['Gene symbol'])
cm = df[l].corr()
import seaborn as sns

#clustermap
sns.clustermap(np.abs(cm),cmap='vlag');
print('Correlations of genes with high rocauc')

In [ ]:
a,b = np.where( np.triu(np.abs(cm),1) > 0.7 )

d = pd.DataFrame()
for i in range(len(a)):
    d.loc[i,'Gene1'] = cm.index[a[i]] 
    d.loc[i,'Gene2'] = cm.index[b[i]] 
    d.loc[i,'Correlation'] = cm.iloc[a[i],b[i]]
    d.loc[i,'Correlation Abs'] = np.abs( cm.iloc[a[i],b[i]] )
    I = np.where(df_genes_alsod['Gene symbol'] == cm.index[a[i]] )[0][0]
    d.loc[i,'Best Abs RocAuc 1'] =  df_genes_alsod['Best Abs RocAuc'].iat[I]
    I = np.where(df_genes_alsod['Gene symbol'] == cm.index[b[i]] )[0][0]
    d.loc[i,'Best Abs RocAuc 2'] =  df_genes_alsod['Best Abs RocAuc'].iat[I]
    
d.sort_values('Correlation Abs',ascending = False).head(30)

In [ ]:
mask = (df_genes_alsod['Best Abs RocAuc'] > 0.0 )
mask.sum()

l = list( df_genes_alsod[mask]['Gene symbol'])
#clustermap
cm149 = df[l].corr()
sns.clustermap(np.abs(cm149),cmap='vlag');
print('Correlations of all genes from the list')

## For comparaison correlations of random data of the same size - it is much LESS correlated 

In [ ]:

random_data_same_size = np.random.randn( df.shape[0], mask.sum() )
random_data_same_size.shape
#clustermap
sns.clustermap(np.abs(np.corrcoef(random_data_same_size.T)),cmap='vlag');
plt.show()

# For comparaison correlation of random subset of the same size 149 of our dataset - again less correlated (not so much less than completely random, but still evidently)

In [ ]:

l = []
for i in range(149):
    j = np.random.randint( df.shape[1] )
    l.append(j)
cm = df.iloc[:,l].corr()    
#clustermap
sns.clustermap(np.abs(cm),cmap='vlag');
plt.show()

In [ ]:
mask = (df_genes_alsod['Best Abs RocAuc'] > 0.0 )
mask.sum()

l = list( df_genes_alsod[mask]['Gene symbol'])
#clustermap
cm149 = df[l].corr()

cm = cm149
v = np.triu((cm),1).ravel()
v = v[v != 0]
plt.hist(v,bins = 100)
plt.title('selected 149 genes data')
plt.show()
print( 'Genes data correlation stastics: ')
print( pd.Series(v).describe() )
print()

random_data_same_size = np.random.randn( df.shape[0], mask.sum() )
random_data_same_size.shape
v = np.triu((np.corrcoef(random_data_same_size.T)),1).ravel()
v = v[v != 0]
plt.hist(v,bins = 100)
plt.title('random  data')
plt.show()
print( 'Random data correlation stastics: ')
print( pd.Series(v).describe() )
print()

print('Random data is much less correlated that these genes data')


l = []
for i in range(149):
    j = np.random.randint( df.shape[1] )
    l.append(j)
cm = df.iloc[:,l].corr()    
v = np.triu((cm),1).ravel()
v = v[v != 0]
plt.hist(v,bins = 100)
plt.title('correlations for random subset of our dataset of the same size 149')
plt.show()
print( 'Correlation stastics for random subset of our dataset of the same size 149 ')
print( pd.Series(v).describe() )
print()
print('Random data is much less correlated that these genes data')


In [ ]:
print('Show top correlated pairs of genes')
mask = (df_genes_alsod['Best Abs RocAuc'] > 0.0 )
mask.sum()

l = list( df_genes_alsod[mask]['Gene symbol'])
#clustermap
cm149 = df[l].corr()


cm = cm149
a,b = np.where( np.triu(np.abs(cm),1) > 0.7 )

d = pd.DataFrame()
for i in range(len(a)):
    d.loc[i,'Gene1'] = cm.index[a[i]] 
    d.loc[i,'Gene2'] = cm.index[b[i]] 
    d.loc[i,'Correlation'] = cm.iloc[a[i],b[i]]
    d.loc[i,'Correlation Abs'] = np.abs( cm.iloc[a[i],b[i]] )
    I = np.where(df_genes_alsod['Gene symbol'] == cm.index[a[i]] )[0][0]
    d.loc[i,'Best Abs RocAuc 1'] =  df_genes_alsod['Best Abs RocAuc'].iat[I]
    I = np.where(df_genes_alsod['Gene symbol'] == cm.index[b[i]] )[0][0]
    d.loc[i,'Best Abs RocAuc 2'] =  df_genes_alsod['Best Abs RocAuc'].iat[I]
    
d.sort_values('Correlation Abs',ascending = False).head(30)

In [ ]:
#f = 'FUS'
#plt.plot(df[f])
print('Look at expression of the genes from the list, we see that some almost not expressed and thus should be thrown out')
for i in range(len(df_genes_alsod)):
    IX = df_genes_alsod.index[i]
    g = df_genes_alsod.loc[IX,'Gene symbol']
    if g in df.columns:
        df_genes_alsod.loc[IX,'Max Expression'] = df[g].max()
        df_genes_alsod.loc[IX,'Mean Expression'] = df[g].mean()
        df_genes_alsod.loc[IX,'Median Expression'] = df[g].median()
        df_genes_alsod.loc[IX,'Min Expression'] = df[g].min()
    
display( df_genes_alsod.describe() )
display(df_genes_alsod.sort_values('Median Expression'))
c = 'Median Expression'
plt.plot( df_genes_alsod[c].sort_values().values , '*-')
plt.show()
print('Count genes with median expression zero:',(df_genes_alsod[c]==0).sum(), 'out of ', len(df_genes_alsod))
c = 'Min Expression'
print('Count genes with min expression zero:',(df_genes_alsod[c]==0).sum(), 'out of ', len(df_genes_alsod))